# Análisis de posibilidad de atención a clientes

Este cuaderno es un análisis de la posibilidad de atender a los clientes de acuerdo a una segmentación dada por un conjunto de condiciones: cumplimiento de 15% de saldo, nivel de RCI, tipo de oferta, etc.

## Condiciones actuales de precio

In [1]:
# importar librerias
import pandas as pd
from utils import cumple_15, calcular_cuota, ingresos_financieros, calcular_monto_a_rci, comparar_montos, reservas, tasa_minima, tasa_maxima

In [2]:
# leer datos
df = pd.read_csv(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\db\creditos.csv")
# renombrar columnas
df.rename(columns={"CODIGO_CLIENTE": "CODIGO CLIENTE",  
                    "outIngresos": "SALARIO",
                    "rciInternoInicialDeudor": "RCI", 
                    "saldo_pr_ext": "SALDO EXTERNO TOTAL CLIENTE"}, inplace=True)

df["TASA"] = df["TASA_ORIGINAL"] / 1200
df["PLAZO"] = df["PLAZO_ORIGINAL"] / 30
# mensualizar la tasa

# tratamiento general de datos
df["CUMPLE 15%"] = df["SALDO"].apply(cumple_15)

# calcular saldo posible a consolidacion
datos_agrupado = df.groupby(by="CODIGO CLIENTE")["SALDO"].sum().reset_index()
datos_agrupado.columns = ["CODIGO CLIENTE", "SALDO INTERNO TOTAL CLIENTE"]
df = pd.merge(df, datos_agrupado, how="left", on="CODIGO CLIENTE")
# sacar saldo total
df["SALDO TOTAL CLIENTE"] = df["SALDO INTERNO TOTAL CLIENTE"] + df["SALDO EXTERNO TOTAL CLIENTE"]


In [4]:
# segmentar clientes con el cumplimiento: segmentar por crédito y luego ver opciones de recrédito agrup
datos = df.copy()

# sacar el monto máximo para llegar a 50% de RCI con la tasa actual más 1
datos["MONTO RCI50 CON TASA ACTUAL +1"] = datos.apply(lambda x: calcular_monto_a_rci(x["TASA"]+1/1200, 50, x["PLAZO"], x["RCI"], x["SALARIO"]), axis=1)
# sacar el motno máximo para llegar a 60% de RCI con tasa actual más 1
datos["MONTO RCI60 CON TASA ACTUAL +1"] = datos.apply(lambda x: calcular_monto_a_rci(x["TASA"]+1/1200, 60, x["PLAZO"], x["RCI"], x["SALARIO"]), axis=1)

# colocar bandera de si se puede consolidar con esa tasa con 50 y 60 de RCI
datos["FLAG CONSOLIDA CON RCI50"] = datos.apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["MONTO RCI50 CON TASA ACTUAL +1"]), axis=1)
datos["FLAG CONSOLIDA CON RCI60"] = datos.apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["MONTO RCI60 CON TASA ACTUAL +1"]), axis=1)


# ingresos a 6 meses con montos máximos aunque no sean consolidaciones
datos["INGRESO A 6 MESES MONTO RCI 50"] = datos.apply(lambda x: ingresos_financieros(x["MONTO RCI50 CON TASA ACTUAL +1"], x["TASA"] + 1/1200, x["PLAZO"], 6), axis=1)
datos["INGRESO A 6 MESES MONTO RCI 60"] = datos.apply(lambda x: ingresos_financieros(x["MONTO RCI60 CON TASA ACTUAL +1"], x["TASA"] + 1/1200, x["PLAZO"], 6), axis=1)

# sacar ingresos a 6 meses solo para los que pueden consolidar a al menos uno de los anteriores
datos["INGRESO A 6 MESES EN CONSOLIDACION"] = datos.apply(lambda x: ingresos_financieros(x["SALDO TOTAL CLIENTE"], x["TASA"] + 1/1200, x["PLAZO"], 6), axis=1)


### Reservas y reestructuración: cálculo con máscaras

In [5]:
mascara_reserva = datos["CUMPLE 15%"] == 0
datos.loc[mascara_reserva, "RESERVAS REESTRUCTURA MONTO RCI50 TASA ACTUAL+1"] = datos.loc[mascara_reserva, "MONTO RCI50 CON TASA ACTUAL +1"].apply(lambda x: reservas(True))
datos.loc[mascara_reserva, "RESERVAS REESTRUCTURA MONTO RCI60 TASA ACTUAL+1"] = datos.loc[mascara_reserva, "MONTO RCI60 CON TASA ACTUAL +1"].apply(lambda x: reservas(True))
datos.loc[mascara_reserva, "RESERVAS CONSOLIDACION TASA ACTUAL +1"] = datos.loc[mascara_reserva, "SALDO TOTAL CLIENTE"].apply(lambda x: reservas(True))

# Compara que clientes cubren reservas en los 6 meses
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS RCI50 TASA ACTUAL +1"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["RESERVAS REESTRUCTURA MONTO RCI50 TASA ACTUAL+1"], x["INGRESO A 6 MESES MONTO RCI 50"]), axis=1)
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS RCI60 TASA ACTUAL +1"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["RESERVAS REESTRUCTURA MONTO RCI60 TASA ACTUAL+1"], x["INGRESO A 6 MESES MONTO RCI 60"]), axis=1)
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS CONSOLIDACION TASA ACTUAL +1"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["INGRESO A 6 MESES EN CONSOLIDACION"]), axis=1)

## Condiciones de precio máximo

In [6]:
# calcular tasa minima para dejar a cero
datos["TASA MINIMA"] = datos.apply(lambda x: tasa_minima(reservas(True)))

# sacar tasa maxima para consolidar
datos["TASA PARA RCI 60"] = datos.apply(lambda x: tasa_maxima(x["RCI"], 60), axis=1)
datos["TASA PARA RCI 50"] = datos.apply(lambda x: tasa_maxima(x["RCI"], 50), axis=1)

# comparar si el intervalo tiene sentido
datos["POSIBLE LIQUIDEZ PRECIO PERSONALIZADO"] = datos.apply(lambda x: comparar_montos(x["TASA MINIMA"], x["TASA PARA RCI 50"]), axis=1)
datos["POSIBLE CONSOLIDACION PRECIO PERSONALIZADO"] = datos.apply(lambda x: comparar_montos(x["TASA MINIMA"], x["TASA PARA RCI 60"]), axis=1)

## Ver si pasan con aumento de plazo

In [7]:
# sacar el monto máximo para llegar a 50% de RCI con la tasa actual más 1
datos["MONTO RCI50 CON TASA ACTUAL +1 Y PLAZO 120 Y TASA ACTUAL +1"] = datos.apply(lambda x: calcular_monto_a_rci(x["TASA"]+1/1200, 50, 120, x["RCI"], x["SALARIO"]), axis=1)
# sacar el motno máximo para llegar a 60% de RCI con tasa actual más 1
datos["MONTO RCI60 CON TASA ACTUAL +1 Y PLAZO 120 Y TASA ACTUAL +1"] = datos.apply(lambda x: calcular_monto_a_rci(x["TASA"]+1/1200, 60, 120, x["RCI"], x["SALARIO"]), axis=1)

# colocar bandera de si se puede consolidar con esa tasa con 50 y 60 de RCI
datos["FLAG CONSOLIDA CON RCI50 Y PLAZO 120 Y TASA ACTUAL +1"] = datos.apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["MONTO RCI50 CON TASA ACTUAL +1 Y PLAZO 120 Y TASA ACTUAL +1"]), axis=1)
datos["FLAG CONSOLIDA CON RCI60 Y PLAZO 120 Y TASA ACTUAL +1"] = datos.apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["MONTO RCI60 CON TASA ACTUAL +1 Y PLAZO 120 Y TASA ACTUAL +1"]), axis=1)


# ingresos a 6 meses con montos máximos aunque no sean consolidaciones
datos["INGRESO A 6 MESES MONTO RCI 50 Y PLAZO 120 Y TASA ACTUAL +1"] = datos.apply(lambda x: ingresos_financieros(x["MONTO RCI50 CON TASA ACTUAL +1 Y PLAZO 120 Y TASA ACTUAL +1"], x["TASA"] + 1/1200, 120, 6), axis=1)
datos["INGRESO A 6 MESES MONTO RCI 60 Y PLAZO 120 Y TASA ACTUAL +1"] = datos.apply(lambda x: ingresos_financieros(x["MONTO RCI60 CON TASA ACTUAL +1 Y PLAZO 120 Y TASA ACTUAL +1"], x["TASA"] + 1/1200, 120, 6), axis=1)

# sacar ingresos a 6 meses solo para los que pueden consolidar a al menos uno de los anteriores
datos["INGRESO A 6 MESES EN CONSOLIDACION Y PLAZO 120 Y TASA ACTUAL +1"] = datos.apply(lambda x: ingresos_financieros(x["SALDO TOTAL CLIENTE"], x["TASA"] + 1/1200, 120, 6), axis=1)


In [12]:
datos.loc[mascara_reserva, "RESERVAS REESTRUCTURA MONTO RCI50 TASA ACTUAL+1 Y PLAZO 120"] = datos.loc[mascara_reserva, "MONTO RCI50 CON TASA ACTUAL +1 Y PLAZO 120 Y TASA ACTUAL +1"].apply(lambda x: reservas(True))
datos.loc[mascara_reserva, "RESERVAS REESTRUCTURA MONTO RCI60 TASA ACTUAL+1 Y PLAZO 120"] = datos.loc[mascara_reserva, "MONTO RCI60 CON TASA ACTUAL +1 Y PLAZO 120 Y TASA ACTUAL +1"].apply(lambda x: reservas(True))
datos.loc[mascara_reserva, "RESERVAS CONSOLIDACION TASA ACTUAL +1 Y PLAZO 120"] = datos.loc[mascara_reserva, "SALDO TOTAL CLIENTE"].apply(lambda x: reservas(True))

# Compara que clientes cubren reservas en los 6 meses
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS RCI50 TASA ACTUAL +1 Y PLAZO 120"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["RESERVAS REESTRUCTURA MONTO RCI50 TASA ACTUAL+1 Y PLAZO 120"], x["INGRESO A 6 MESES MONTO RCI 50 Y PLAZO 120 Y TASA ACTUAL +1"]), axis=1)
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS RCI60 TASA ACTUAL +1 Y PLAZO 120"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["RESERVAS REESTRUCTURA MONTO RCI60 TASA ACTUAL+1 Y PLAZO 120"], x["INGRESO A 6 MESES MONTO RCI 60 Y PLAZO 120 Y TASA ACTUAL +1"]), axis=1)
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS CONSOLIDACION TASA ACTUAL +1 Y PLAZO 120"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["INGRESO A 6 MESES EN CONSOLIDACION Y PLAZO 120 Y TASA ACTUAL +1"]), axis=1)


## Plazo 144

In [13]:
# sacar el monto máximo para llegar a 50% de RCI con la tasa actual más 1
datos["MONTO RCI50 CON TASA ACTUAL +1 Y PLAZO 144"] = datos.apply(lambda x: calcular_monto_a_rci(x["TASA"]+1/1200, 50, 144, x["RCI"], x["SALARIO"]), axis=1)
# sacar el motno máximo para llegar a 60% de RCI con tasa actual más 1
datos["MONTO RCI60 CON TASA ACTUAL +1 Y PLAZO 144"] = datos.apply(lambda x: calcular_monto_a_rci(x["TASA"]+1/1200, 60, 144, x["RCI"], x["SALARIO"]), axis=1)

# colocar bandera de si se puede consolidar con esa tasa con 50 y 60 de RCI
datos["FLAG CONSOLIDA CON RCI50 Y PLAZO 144 TASA ACTUAL +1"] = datos.apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["MONTO RCI50 CON TASA ACTUAL +1 Y PLAZO 144"]), axis=1)
datos["FLAG CONSOLIDA CON RCI60 Y PLAZO 144 TASA ACTUAL +1"] = datos.apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["MONTO RCI60 CON TASA ACTUAL +1 Y PLAZO 144"]), axis=1)


# ingresos a 6 meses con montos máximos aunque no sean consolidaciones
datos["INGRESO A 6 MESES MONTO RCI 50 Y PLAZO 144"] = datos.apply(lambda x: ingresos_financieros(x["MONTO RCI50 CON TASA ACTUAL +1 Y PLAZO 144"], x["TASA"] + 1/1200, 144, 6), axis=1)
datos["INGRESO A 6 MESES MONTO RCI 60 Y PLAZO 144"] = datos.apply(lambda x: ingresos_financieros(x["MONTO RCI60 CON TASA ACTUAL +1 Y PLAZO 144"], x["TASA"] + 1/1200, 144, 6), axis=1)

# sacar ingresos a 6 meses solo para los que pueden consolidar a al menos uno de los anteriores
datos["INGRESO A 6 MESES EN CONSOLIDACION Y PLAZO 144"] = datos.apply(lambda x: ingresos_financieros(x["SALDO TOTAL CLIENTE"], x["TASA"] + 1/1200, 144, 6), axis=1)


In [14]:
datos.loc[mascara_reserva, "RESERVAS REESTRUCTURA MONTO RCI50 TASA ACTUAL+1 Y PLAZO 144"] = datos.loc[mascara_reserva, "MONTO RCI50 CON TASA ACTUAL +1 Y PLAZO 144"].apply(lambda x: reservas(True))
datos.loc[mascara_reserva, "RESERVAS REESTRUCTURA MONTO RCI60 TASA ACTUAL+1 Y PLAZO 144"] = datos.loc[mascara_reserva, "MONTO RCI60 CON TASA ACTUAL +1 Y PLAZO 144"].apply(lambda x: reservas(True))
datos.loc[mascara_reserva, "RESERVAS CONSOLIDACION TASA ACTUAL +1 Y PLAZO 144"] = datos.loc[mascara_reserva, "SALDO TOTAL CLIENTE"].apply(lambda x: reservas(True))

# Compara que clientes cubren reservas en los 6 meses
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS RCI50 TASA ACTUAL +1 Y PLAZO 144"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["RESERVAS REESTRUCTURA MONTO RCI50 TASA ACTUAL+1 Y PLAZO 144"], x["INGRESO A 6 MESES MONTO RCI 50 Y PLAZO 144"]), axis=1)
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS RCI60 TASA ACTUAL +1 Y PLAZO 144"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["RESERVAS REESTRUCTURA MONTO RCI60 TASA ACTUAL+1 Y PLAZO 144"], x["INGRESO A 6 MESES MONTO RCI 60 Y PLAZO 144"]), axis=1)
datos.loc[mascara_reserva, "CUMPLE RESERVAS CON INGRESOS CONSOLIDACION TASA ACTUAL +1 Y PLAZO 144"] = datos.loc[mascara_reserva].apply(lambda x: comparar_montos(x["SALDO TOTAL CLIENTE"], x["INGRESO A 6 MESES EN CONSOLIDACION Y PLAZO 144"]), axis=1)


In [15]:
# exportar
datos.to_csv(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\comerciales\inhabilitacion.csv")